##Solución

Se realiza la iteración sobre el dataset y se va extrayendo de la wikipedia la descripción en base a la columna full_name.
Primeramente se realiza una comprobación de si en la respuesta de la wikipedia existe o no una descripción de ese personaje. Si no existe, no se almacena en el nuevo dataset.

Esta celda de código tardará en ejecutarse debido a la cantidad de peticiones a la api de Wikipedia. Puede ir viéndose el progreso en el archivo "logExtractBiographies.txt" que se crea el propio script

In [14]:
import pandas as pd
import requests
import sys

originalData = pd.read_csv("pantheon1database.csv")

#Este objeto logger se emplea únicamente para ir guardando en un fichero un registro de las tuplas que vamos procesando. (Por si sucede algún error)
logger= open("logExtractBiographies.txt","w")
logger.write("Log for Extract Bios\n")
logger.close();

#Dataset en el que se van a ir añadiendo las descripciones asociadas a los nombres
finalDataset = pd.DataFrame(columns = ['full_name', 'wiki_description'])

actualTuple = 1
numTuples = originalData.shape[0]

for row in originalData.itertuples():
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"
    
    #row[1] contiene el id del articulo en wikipedia. row[2] el nombre del personaje histórico
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": row[2],
        "prop": "extracts",
        "explaintext": "1",
        "exintro": "1"
    }
    try:
        
        R = S.get(url=URL, params=PARAMS)

        DATA = R.json()
        if "extract" in DATA['query']['pages'][list(DATA['query']['pages'].keys())[0]]:
            wiki_description = DATA['query']['pages'][list(DATA['query']['pages'].keys())[0]]['extract']
            if (wiki_description ==""):
                wiki_description=None
        else:
            wiki_description =None
        #añadimos la tupla con el nombre y la descripción al dataset si es que existe
        if (wiki_description != None):
          finalDataset = finalDataset.append({'full_name':row[2], 'wiki_description':wiki_description}, ignore_index=True)

        logger= open("logExtractBiographies.txt","a")
        logger.write("Retrieved wiki about "+ str(row[2])+ " "+str(actualTuple)+ " / "+str(numTuples)+"\n")
        logger.close()
        
    except Exception as e:
        logger= open("logExtractBiographies.txt","a")
        logger.write("Oops!"+ str(e.__class__)+ " occurred."+"\n")
        logger.close()
        
    actualTuple = actualTuple+1

#guardamos en un csv el nuevo dataset
finalDataset.to_csv("pantheonWikipediaData.csv",  index=False)

    


Si queremos hacer el join de los dos dataset (el original y el nuevo que contiene las descripciones):

In [15]:
newData = pd.merge(originalData, finalDataset, on="full_name")
newData.to_csv("pantheonDatabaseConBiografias.csv",  index=False)